In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import to_date
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime
from pyspark.sql.types import DoubleType, IntegerType, DateType, TimestampType


### Creating APP

In [2]:
spark = SparkSession.builder \
    .master("local") \
    .appName("PySpark SQL Table Joining") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

### Reading dataset

In [3]:
#Initializing File Type and path for data train
file_type = 'text'
delimeter=','


In [4]:
def load_data(file_type):
    """input type of file "text" or "parquet" and Return pyspark dataframe"""
    if file_type =="text": # use text as file type input
        df = spark.read.option("header", "true") \
                       .option("delimeter",delimeter)\
                       .option("inferSchema", "true") \
                       .csv(path)  #path file that you want import
    else:  
        df= spark.read.parquet("example.parquet") #path file that you want import
    return df


In [5]:
# reading worker dataset
path=r'/home/hasan/DATA SET/PySpark SQL/Worker.csv'
worker = load_data(file_type)

# reading title dataset
path = r'/home/hasan/DATA SET/PySpark SQL/Title.csv'
title = load_data(file_type)

# reading bonus dataset
path = r'/home/hasan/DATA SET/PySpark SQL/Bonus.csv'
bonus = load_data(file_type)


### converting to DateType

In [6]:
worker = worker.withColumn("JOINING_DATE", worker["JOINING_DATE"].cast(DateType()))
title = title.withColumn("AFFECTED_FROM", title["AFFECTED_FROM"].cast(DateType()))
bonus = bonus.withColumn("BONUS_DATE", bonus["BONUS_DATE"].cast(DateType()))


### Initializing Dataset

In [7]:
worker.show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|
|        2|  Niharika|    Verma| 80000|  2014-08-24|     Admin|
|        3|    Vishal|  Singhal|300000|  2014-02-16|        HR|
|        4|   Amitabh|    Singh|500000|  2014-05-09|     Admin|
|        5|     Vivek|    Bhati|500000|  2014-07-12|     Admin|
|        6|     Vipul|    Diwan|200000|  2014-11-16|   Account|
|        7|    Satish|    Kumar| 75000|  2014-09-19|   Account|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
|        9|     Subro|      Roy| 75000|  2014-10-21|   Account|
|       10|  Soumitro|      Das| 84000|  2014-04-13|     Admin|
+---------+----------+---------+------+------------+----------+



In [8]:
title.show()

+-------------+-------------+-------------+
|WORKER_REF_ID| WORKER_TITLE|AFFECTED_FROM|
+-------------+-------------+-------------+
|            1|      Manager|   2016-02-20|
|            2|    Executive|   2016-06-11|
|            8|    Executive|   2016-06-11|
|           10|         Lead|   2016-06-20|
|            5|      Manager|   2016-06-11|
|            4|Asst. Manager|   2016-06-11|
|            9|      Manager|   2016-03-20|
|            7|    Executive|   2016-06-11|
|            6|         Lead|   2016-06-11|
|            3|         Lead|   2016-06-11|
+-------------+-------------+-------------+



In [9]:
bonus.show()

+-------------+----------+------------+
|WORKER_REF_ID|BONUS_DATE|BONUS_AMOUNT|
+-------------+----------+------------+
|            1|2014-07-25|        5000|
|            2|2014-08-24|        3000|
|            3|2014-02-16|        4000|
|            1|2014-07-25|        4500|
|            2|2014-08-24|        3500|
+-------------+----------+------------+



In [10]:
worker.printSchema()

root
 |-- WORKER_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- JOINING_DATE: date (nullable = true)
 |-- DEPARTMENT: string (nullable = true)



In [11]:
title.printSchema()

root
 |-- WORKER_REF_ID: integer (nullable = true)
 |-- WORKER_TITLE: string (nullable = true)
 |-- AFFECTED_FROM: date (nullable = true)



In [12]:
bonus.printSchema()

root
 |-- WORKER_REF_ID: integer (nullable = true)
 |-- BONUS_DATE: date (nullable = true)
 |-- BONUS_AMOUNT: integer (nullable = true)



### Converting Dataset to SQL Query

In [13]:
# Register the DataFrame as a SQL temporary view
worker.createOrReplaceTempView("worker_data")
title.createOrReplaceTempView("title_data")
bonus.createOrReplaceTempView("bonus_data")

### Question and Answer

In [14]:
#Q-1. Write an SQL query to fetch “FIRST_NAME” from Worker table using the alias name as <WORKER_NAME>.
spark.sql(" select FIRST_NAME as WORKER_NAME from worker_data ").show()

+-----------+
|WORKER_NAME|
+-----------+
|     Monika|
|   Niharika|
|     Vishal|
|    Amitabh|
|      Vivek|
|      Vipul|
|     Satish|
|    Geetika|
|      Subro|
|   Soumitro|
+-----------+



In [15]:
#Q-2. Write an SQL query to fetch “FIRST_NAME” from Worker table in upper case.
spark.sql(" select upper(FIRST_NAME) from worker_data ").show()

+-----------------+
|upper(FIRST_NAME)|
+-----------------+
|           MONIKA|
|         NIHARIKA|
|           VISHAL|
|          AMITABH|
|            VIVEK|
|            VIPUL|
|           SATISH|
|          GEETIKA|
|            SUBRO|
|         SOUMITRO|
+-----------------+



In [16]:
#Q-3. Write an SQL query to fetch unique values of DEPARTMENT from Worker table.
spark.sql(" select distinct(DEPARTMENT) from worker_data ").show()

+----------+
|DEPARTMENT|
+----------+
|        HR|
|     Admin|
|   Account|
+----------+



In [17]:
# Q-4. Write an SQL query to print the first three characters of  FIRST_NAME from Worker table.
spark.sql(" select substring(FIRST_NAME,1,3) from worker_data ").show()

+---------------------------+
|substring(FIRST_NAME, 1, 3)|
+---------------------------+
|                        Mon|
|                        Nih|
|                        Vis|
|                        Ami|
|                        Viv|
|                        Vip|
|                        Sat|
|                        Gee|
|                        Sub|
|                        Sou|
+---------------------------+



In [18]:
# Q-5. Write an SQL query to find the position of the alphabet (‘a’) in the first name column ‘Amitabh’ from Worker table.
spark.sql(" Select instr(FIRST_NAME, 'a') from worker_data where FIRST_NAME = 'Amitabh' ").show()

+--------------------+
|instr(FIRST_NAME, a)|
+--------------------+
|                   5|
+--------------------+



In [19]:
# Q-6. Write an SQL query to print the FIRST_NAME from Worker table after removing white spaces from the right side.
spark.sql(" Select RTRIM(FIRST_NAME) from worker_data ").show()

+-----------------+
|rtrim(FIRST_NAME)|
+-----------------+
|           Monika|
|         Niharika|
|           Vishal|
|          Amitabh|
|            Vivek|
|            Vipul|
|           Satish|
|          Geetika|
|            Subro|
|         Soumitro|
+-----------------+



In [20]:
# Q-7. Write an SQL query to print the DEPARTMENT from Worker table after removing white spaces from the left side.
spark.sql(" select ltrim(DEPARTMENT) from worker_data ").show()

+-----------------+
|ltrim(DEPARTMENT)|
+-----------------+
|               HR|
|            Admin|
|               HR|
|            Admin|
|            Admin|
|          Account|
|          Account|
|            Admin|
|          Account|
|            Admin|
+-----------------+



In [21]:
# Q-8. Write an SQL query that fetches the unique values of DEPARTMENT from Worker table and prints its length.
spark.sql(" select distinct length(DEPARTMENT) from worker_data ").show()

+------------------+
|length(DEPARTMENT)|
+------------------+
|                 5|
|                 7|
|                 2|
+------------------+



In [22]:
# Q-9. Write an SQL query to print the FIRST_NAME from Worker table after replacing ‘a’ with ‘A’.
spark.sql(" select replace(FIRST_NAME, 'a', 'A') from worker_data").show()

+-------------------------+
|replace(FIRST_NAME, a, A)|
+-------------------------+
|                   MonikA|
|                 NihArikA|
|                   VishAl|
|                  AmitAbh|
|                    Vivek|
|                    Vipul|
|                   SAtish|
|                  GeetikA|
|                    Subro|
|                 Soumitro|
+-------------------------+



In [23]:
# Q-10. Write an SQL query to print the FIRST_NAME and LAST_NAME from Worker table into a single column COMPLETE_NAME. A space char should separate them.
spark.sql(" select concat(FIRST_NAME, ' ', LAST_NAME) as COMPLETE_NAME from worker_data ").show()

+---------------+
|  COMPLETE_NAME|
+---------------+
|   Monika Arora|
| Niharika Verma|
| Vishal Singhal|
|  Amitabh Singh|
|    Vivek Bhati|
|    Vipul Diwan|
|   Satish Kumar|
|Geetika Chauhan|
|      Subro Roy|
|   Soumitro Das|
+---------------+



In [24]:
# Q-11. Write an SQL query to print all Worker details from the Worker table order by FIRST_NAME Ascending.
spark.sql(" select * from worker_data order by FIRST_NAME asc ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        4|   Amitabh|    Singh|500000|  2014-05-09|     Admin|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|
|        2|  Niharika|    Verma| 80000|  2014-08-24|     Admin|
|        7|    Satish|    Kumar| 75000|  2014-09-19|   Account|
|       10|  Soumitro|      Das| 84000|  2014-04-13|     Admin|
|        9|     Subro|      Roy| 75000|  2014-10-21|   Account|
|        6|     Vipul|    Diwan|200000|  2014-11-16|   Account|
|        3|    Vishal|  Singhal|300000|  2014-02-16|        HR|
|        5|     Vivek|    Bhati|500000|  2014-07-12|     Admin|
+---------+----------+---------+------+------------+----------+



In [25]:
# Q-12. Write an SQL query to print all Worker details from the Worker table order by FIRST_NAME Ascending and DEPARTMENT Descending.
spark.sql(" select * from worker_data order by FIRST_NAME asc, DEPARTMENT desc ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        4|   Amitabh|    Singh|500000|  2014-05-09|     Admin|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|
|        2|  Niharika|    Verma| 80000|  2014-08-24|     Admin|
|        7|    Satish|    Kumar| 75000|  2014-09-19|   Account|
|       10|  Soumitro|      Das| 84000|  2014-04-13|     Admin|
|        9|     Subro|      Roy| 75000|  2014-10-21|   Account|
|        6|     Vipul|    Diwan|200000|  2014-11-16|   Account|
|        3|    Vishal|  Singhal|300000|  2014-02-16|        HR|
|        5|     Vivek|    Bhati|500000|  2014-07-12|     Admin|
+---------+----------+---------+------+------------+----------+



In [26]:
# Q-13. Write an SQL query to print details for Workers with the first name as “Vipul” and “Satish” from Worker table.
spark.sql(" select * from worker_data where FIRST_NAME in ('Vipul', 'Satish') ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        6|     Vipul|    Diwan|200000|  2014-11-16|   Account|
|        7|    Satish|    Kumar| 75000|  2014-09-19|   Account|
+---------+----------+---------+------+------------+----------+



In [27]:
# Q-14. Write an SQL query to print details of workers excluding first names, “Vipul” and “Satish” from Worker table.
spark.sql(" select * from worker_data where FIRST_NAME not in ('Vipul', 'Satish') ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|
|        2|  Niharika|    Verma| 80000|  2014-08-24|     Admin|
|        3|    Vishal|  Singhal|300000|  2014-02-16|        HR|
|        4|   Amitabh|    Singh|500000|  2014-05-09|     Admin|
|        5|     Vivek|    Bhati|500000|  2014-07-12|     Admin|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
|        9|     Subro|      Roy| 75000|  2014-10-21|   Account|
|       10|  Soumitro|      Das| 84000|  2014-04-13|     Admin|
+---------+----------+---------+------+------------+----------+



In [28]:
# Q-15. Write an SQL query to print details of Workers with DEPARTMENT name as “Admin”.
spark.sql(" select * from worker_data where DEPARTMENT='Admin' ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        2|  Niharika|    Verma| 80000|  2014-08-24|     Admin|
|        4|   Amitabh|    Singh|500000|  2014-05-09|     Admin|
|        5|     Vivek|    Bhati|500000|  2014-07-12|     Admin|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
|       10|  Soumitro|      Das| 84000|  2014-04-13|     Admin|
+---------+----------+---------+------+------------+----------+



In [29]:
# Q-16. Write an SQL query to print details of the Workers whose FIRST_NAME contains ‘a’.
spark.sql(" select * from worker_data where FIRST_NAME like '%a%' ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|
|        2|  Niharika|    Verma| 80000|  2014-08-24|     Admin|
|        3|    Vishal|  Singhal|300000|  2014-02-16|        HR|
|        4|   Amitabh|    Singh|500000|  2014-05-09|     Admin|
|        7|    Satish|    Kumar| 75000|  2014-09-19|   Account|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
+---------+----------+---------+------+------------+----------+



In [30]:
# Q-17. Write an SQL query to print details of the Workers whose FIRST_NAME ends with ‘a’.
spark.sql(" select * from worker_data where FIRST_NAME like '%a' ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|
|        2|  Niharika|    Verma| 80000|  2014-08-24|     Admin|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
+---------+----------+---------+------+------------+----------+



In [31]:
# Q-18. Write an SQL query to print details of the Workers whose FIRST_NAME ends with ‘h’ and contains six alphabets.
spark.sql(" select * from worker_data where FIRST_NAME like '_____h' ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        7|    Satish|    Kumar| 75000|  2014-09-19|   Account|
+---------+----------+---------+------+------------+----------+



In [32]:
# Q-19. Write an SQL query to print details of the Workers whose SALARY lies between 100000 and 500000.
spark.sql(" select * from worker_data where SALARY between 100000 and 500000 ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|
|        3|    Vishal|  Singhal|300000|  2014-02-16|        HR|
|        4|   Amitabh|    Singh|500000|  2014-05-09|     Admin|
|        5|     Vivek|    Bhati|500000|  2014-07-12|     Admin|
|        6|     Vipul|    Diwan|200000|  2014-11-16|   Account|
+---------+----------+---------+------+------------+----------+



In [64]:
# Q-20. Write an SQL query to print details of the Workers who have joined in Feb’2014.
spark.sql(" select * from worker_data where year(JOINING_DATE)=2014 and month(JOINING_DATE)=2 ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        3|    Vishal|  Singhal|300000|  2014-02-16|        HR|
+---------+----------+---------+------+------------+----------+



In [34]:
# Q-21. Write an SQL query to fetch the count of employees working in the department ‘Admin’.
spark.sql(" select count(*) from worker_data where DEPARTMENT='Admin' ").show()

+--------+
|count(1)|
+--------+
|       5|
+--------+



In [35]:
# Q-22. Write an SQL query to fetch worker names with salaries >= 50000 and <= 100000.
spark.sql(" select FIRST_NAME from worker_data where SALARY>=50000 and SALARY<=100000 ").show()

+----------+
|FIRST_NAME|
+----------+
|    Monika|
|  Niharika|
|    Satish|
|   Geetika|
|     Subro|
|  Soumitro|
+----------+



In [36]:
# Q-23. Write an SQL query to fetch the no. of workers for each department in the descending order.
spark.sql(" select DEPARTMENT,  count(WORKER_ID) no_worker from worker_data group by DEPARTMENT order by no_worker desc ").show()

+----------+---------+
|DEPARTMENT|no_worker|
+----------+---------+
|     Admin|        5|
|   Account|        3|
|        HR|        2|
+----------+---------+



In [37]:
# Q-23. Write an SQL query to fetch the no. of workers for each department in the descending order.
spark.sql(" select count(DEPARTMENT) no_worker from worker_data group by DEPARTMENT order by no_worker desc ").show()

+---------+
|no_worker|
+---------+
|        5|
|        3|
|        2|
+---------+



In [38]:
# Q-24. Write an SQL query to print details of the Workers who are also Managers.
spark.sql(" select * from worker_data w inner join title_data t on w.WORKER_ID=t.WORKER_REF_ID and t.WORKER_TITLE='Manager' ").show()

+---------+----------+---------+------+------------+----------+-------------+------------+-------------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|WORKER_REF_ID|WORKER_TITLE|AFFECTED_FROM|
+---------+----------+---------+------+------------+----------+-------------+------------+-------------+
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|            1|     Manager|   2016-02-20|
|        5|     Vivek|    Bhati|500000|  2014-07-12|     Admin|            5|     Manager|   2016-06-11|
|        9|     Subro|      Roy| 75000|  2014-10-21|   Account|            9|     Manager|   2016-03-20|
+---------+----------+---------+------+------------+----------+-------------+------------+-------------+



In [39]:
# Q-24. Write an SQL query to print details of the Workers who are also Managers.
spark.sql(" select w.FIRST_NAME, T.WORKER_TITLE from worker_data w inner join title_data t on w.WORKER_ID=t.WORKER_REF_ID and t.WORKER_TITLE='Manager' ").show()

+----------+------------+
|FIRST_NAME|WORKER_TITLE|
+----------+------------+
|    Monika|     Manager|
|     Vivek|     Manager|
|     Subro|     Manager|
+----------+------------+



In [40]:
# Q-25. Write an SQL query to fetch duplicate records having matching data in some fields of a table.
spark.sql(" select WORKER_TITLE, AFFECTED_FROM, count(*) from title_data group by WORKER_TITLE, AFFECTED_FROM having count(*) > 1 ").show()

+------------+-------------+--------+
|WORKER_TITLE|AFFECTED_FROM|count(1)|
+------------+-------------+--------+
|   Executive|   2016-06-11|       3|
|        Lead|   2016-06-11|       2|
+------------+-------------+--------+



In [41]:
# Q-25 Write and SQL query to find department name which are appear more than 1 times.
spark.sql(" SELECT DEPARTMENT, COUNT(*) FROM worker_data GROUP BY DEPARTMENT HAVING COUNT(*) > 1 ").show()

+----------+--------+
|DEPARTMENT|count(1)|
+----------+--------+
|        HR|       2|
|     Admin|       5|
|   Account|       3|
+----------+--------+



In [42]:
# Q-26. Write an SQL query to show only odd rows from a table.
spark.sql(" select * from worker_data where mod(WORKER_ID, 2)<>0 ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|
|        3|    Vishal|  Singhal|300000|  2014-02-16|        HR|
|        5|     Vivek|    Bhati|500000|  2014-07-12|     Admin|
|        7|    Satish|    Kumar| 75000|  2014-09-19|   Account|
|        9|     Subro|      Roy| 75000|  2014-10-21|   Account|
+---------+----------+---------+------+------------+----------+



In [43]:
# Q-27. Write an SQL query to show only even rows from a table.
spark.sql(" select * from worker_data where mod(WORKER_ID, 2)=0 ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        2|  Niharika|    Verma| 80000|  2014-08-24|     Admin|
|        4|   Amitabh|    Singh|500000|  2014-05-09|     Admin|
|        6|     Vipul|    Diwan|200000|  2014-11-16|   Account|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
|       10|  Soumitro|      Das| 84000|  2014-04-13|     Admin|
+---------+----------+---------+------+------------+----------+



#### Below two question is not possible because those table have date column

In [ ]:
#Q-29. Write an SQL query to fetch intersecting records of two tables.
spark.sql(" (SELECT * FROM bonus_data) INTERSECT (SELECT * FROM title_data) ").show()

In [ ]:
# Q-30. Write an SQL query to show records from one table that another table does not have.
spark.sql(" (select * from bonus_data) minus (select * from title_data) ").show()

In [44]:
# Q-31. Write an SQL query to show the current date and time.
spark.sql(" SELECT NOW() ").show()

+--------------------+
|               now()|
+--------------------+
|2020-07-27 22:48:...|
+--------------------+



In [45]:
# Q-32. Write an SQL query to show the top n (say 10) records of a table.
spark.sql(" select * from worker_data order by SALARY limit 5").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        7|    Satish|    Kumar| 75000|  2014-09-19|   Account|
|        9|     Subro|      Roy| 75000|  2014-10-21|   Account|
|        2|  Niharika|    Verma| 80000|  2014-08-24|     Admin|
|       10|  Soumitro|      Das| 84000|  2014-04-13|     Admin|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
+---------+----------+---------+------+------------+----------+



In [46]:
# Q-33. Write an SQL query to determine the nth (say n=5) highest salary from a table.
spark.sql(" select SALARY from worker_data order by SALARY limit 5").show()

+------+
|SALARY|
+------+
| 75000|
| 75000|
| 80000|
| 84000|
| 90000|
+------+



In [47]:
# Q-35. Write an SQL query to fetch the list of employees with the same salary.
spark.sql(" select distinct W.WORKER_ID, W.FIRST_NAME, W.SALARY from worker_data W, worker_data W1 where W.SALARY=W1.SALARY and W.WORKER_ID != W1.WORKER_ID ").show()


+---------+----------+------+
|WORKER_ID|FIRST_NAME|SALARY|
+---------+----------+------+
|        7|    Satish| 75000|
|        9|     Subro| 75000|
|        5|     Vivek|500000|
|        4|   Amitabh|500000|
+---------+----------+------+



In [48]:
# Q-36. Write an SQL query to show the second highest salary from a table.
spark.sql(" select max(SALARY) from worker_data where SALARY not in (select max(SALARY) from worker_data) ").show()

+-----------+
|max(SALARY)|
+-----------+
|     300000|
+-----------+



In [49]:
# Q-37. Write an SQL query to show one row twice in results from a table.
spark.sql(" (select FIRST_NAME, SALARY from worker_data W1) union all (select FIRST_NAME, SALARY from worker_data W2) ").show()

+----------+------+
|FIRST_NAME|SALARY|
+----------+------+
|    Monika|100000|
|  Niharika| 80000|
|    Vishal|300000|
|   Amitabh|500000|
|     Vivek|500000|
|     Vipul|200000|
|    Satish| 75000|
|   Geetika| 90000|
|     Subro| 75000|
|  Soumitro| 84000|
|    Monika|100000|
|  Niharika| 80000|
|    Vishal|300000|
|   Amitabh|500000|
|     Vivek|500000|
|     Vipul|200000|
|    Satish| 75000|
|   Geetika| 90000|
|     Subro| 75000|
|  Soumitro| 84000|
+----------+------+



#### below question is not possible because table have date column

In [ ]:
# Q-38. Write an SQL query to fetch intersecting records of two tables.
spark.sql(" (select * from title_data) intersect (select * from bonus_data) ").show()

In [50]:
# Q-39. Write an SQL query to fetch the first 50% records from a table.
spark.sql(" select * from worker_data where WORKER_ID > (select count(WORKER_ID)/2 from worker_data) ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        6|     Vipul|    Diwan|200000|  2014-11-16|   Account|
|        7|    Satish|    Kumar| 75000|  2014-09-19|   Account|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
|        9|     Subro|      Roy| 75000|  2014-10-21|   Account|
|       10|  Soumitro|      Das| 84000|  2014-04-13|     Admin|
+---------+----------+---------+------+------------+----------+



In [51]:
# Q-40. Write an SQL query to fetch the departments that have less than five people in it.
spark.sql(" select DEPARTMENT, count(WORKER_ID) as number_of_worker from worker_data group by DEPARTMENT having count(WORKER_ID)< 5 ").show()


+----------+----------------+
|DEPARTMENT|number_of_worker|
+----------+----------------+
|        HR|               2|
|   Account|               3|
+----------+----------------+



In [52]:
# Q-40. Write an SQL query to fetch the departments that have greater than four people in it.
spark.sql(" SELECT DEPARTMENT, COUNT(WORKER_ID) as Number_of_Workers FROM worker_data GROUP BY DEPARTMENT HAVING COUNT(WORKER_ID) >= 5 ").show()

+----------+-----------------+
|DEPARTMENT|Number_of_Workers|
+----------+-----------------+
|     Admin|                5|
+----------+-----------------+



In [53]:
# Q-41. Write an SQL query to show all departments along with the number of people in there.
spark.sql(" select DEPARTMENT, count(DEPARTMENT) as number_employee from worker_data group by DEPARTMENT ").show()

+----------+---------------+
|DEPARTMENT|number_employee|
+----------+---------------+
|        HR|              2|
|     Admin|              5|
|   Account|              3|
+----------+---------------+



In [54]:
# Q-42. Write an SQL query to show the last record from a table.
spark.sql(" select * from worker_data where WORKER_ID = (select max(WORKER_ID) from worker_data) ").show()



+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|       10|  Soumitro|      Das| 84000|  2014-04-13|     Admin|
+---------+----------+---------+------+------------+----------+



In [55]:
# Q-43. Write an SQL query to fetch the first row of a table.
spark.sql(" select * from worker_data where WORKER_ID=(select min(WORKER_ID) from worker_data) ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|
+---------+----------+---------+------+------------+----------+



In [56]:
# Q-44. Write an SQL query to fetch the last five records from a table.
spark.sql(" select * from worker_data where WORKER_ID>=5 ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        5|     Vivek|    Bhati|500000|  2014-07-12|     Admin|
|        6|     Vipul|    Diwan|200000|  2014-11-16|   Account|
|        7|    Satish|    Kumar| 75000|  2014-09-19|   Account|
|        8|   Geetika|  Chauhan| 90000|  2014-03-22|     Admin|
|        9|     Subro|      Roy| 75000|  2014-10-21|   Account|
|       10|  Soumitro|      Das| 84000|  2014-04-13|     Admin|
+---------+----------+---------+------+------------+----------+



In [57]:
# Q-44. Write an SQL query to fetch the last five records from a table.
spark.sql(" SELECT * FROM worker_data WHERE WORKER_ID <=5 UNION SELECT * FROM (SELECT * FROM worker_data W order by W.WORKER_ID DESC) AS W1 WHERE W1.WORKER_ID <=5 ").show()

+---------+----------+---------+------+------------+----------+
|WORKER_ID|FIRST_NAME|LAST_NAME|SALARY|JOINING_DATE|DEPARTMENT|
+---------+----------+---------+------+------------+----------+
|        5|     Vivek|    Bhati|500000|  2014-07-12|     Admin|
|        2|  Niharika|    Verma| 80000|  2014-08-24|     Admin|
|        3|    Vishal|  Singhal|300000|  2014-02-16|        HR|
|        4|   Amitabh|    Singh|500000|  2014-05-09|     Admin|
|        1|    Monika|    Arora|100000|  2014-07-25|        HR|
+---------+----------+---------+------+------------+----------+



In [58]:
# Q-45. Write an SQL query to print the name of employees having the highest salary in each department.
spark.sql(" select FIRST_NAME, DEPARTMENT, SALARY from worker_data where SALARY in (select max(SALARY) as salary from worker_data group by DEPARTMENT) ").show()

+----------+----------+------+
|FIRST_NAME|DEPARTMENT|SALARY|
+----------+----------+------+
|    Vishal|        HR|300000|
|   Amitabh|     Admin|500000|
|     Vivek|     Admin|500000|
|     Vipul|   Account|200000|
+----------+----------+------+



In [59]:
# Q-46. Write an SQL query to fetch three max salaries from a table.
spark.sql(" select FIRST_NAME, SALARY from worker_data order by SALARY desc limit 3 ").show()


+----------+------+
|FIRST_NAME|SALARY|
+----------+------+
|   Amitabh|500000|
|     Vivek|500000|
|    Vishal|300000|
+----------+------+



In [60]:
# Q-47. Write an SQL query to fetch three min salaries from a table.
spark.sql(" select FIRST_NAME, SALARY from worker_data order by SALARY asc limit 3 ").show()

+----------+------+
|FIRST_NAME|SALARY|
+----------+------+
|    Satish| 75000|
|     Subro| 75000|
|  Niharika| 80000|
+----------+------+



In [61]:
# Q-48. Write an SQL query to fetch nth max salaries from a table.
spark.sql(" select FIRST_NAME, SALARY from worker_data order by SALARY desc limit 5").show()

+----------+------+
|FIRST_NAME|SALARY|
+----------+------+
|   Amitabh|500000|
|     Vivek|500000|
|    Vishal|300000|
|     Vipul|200000|
|    Monika|100000|
+----------+------+



In [62]:
# Q-49. Write an SQL query to fetch departments along with the total salaries paid for each of them.
spark.sql(" select DEPARTMENT, sum(SALARY) from worker_data group by DEPARTMENT ").show()


+----------+-----------+
|DEPARTMENT|sum(SALARY)|
+----------+-----------+
|        HR|     400000|
|     Admin|    1254000|
|   Account|     350000|
+----------+-----------+



In [63]:
# Q-50. Write an SQL query to fetch the names of workers who earn the highest salary.
spark.sql(" select FIRST_NAME, SALARY from worker_data where SALARY=(select max(SALARY) from worker_data) ").show()


+----------+------+
|FIRST_NAME|SALARY|
+----------+------+
|   Amitabh|500000|
|     Vivek|500000|
+----------+------+

